In [583]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [584]:
def getData(type):
    driver = webdriver.Chrome(r"C:\Users\jleus\chromedriver.exe")
    driver.get("https://www.actionnetwork.com/odds")
    if type == 'total':
        driver.find_element_by_xpath('/html/body/div/div/main/div[2]/div[2]/div[1]/div/div[2]/select/option[2]').click()
    elif type == 'moneyline':
        driver.find_element_by_xpath('/html/body/div/div/main/div[2]/div[2]/div[1]/div/div[2]/select/option[3]').click()
    rows = driver.find_elements_by_tag_name('tr')
    return rows

In [585]:
def convertToSoup(rows):
    soup = []
    for row in rows:
        elementHTML = row.get_attribute('outerHTML')
        soup.append(BeautifulSoup(elementHTML, 'lxml'))
    return soup


In [586]:
def findData(soup):
    rows = []
    for tr in soup:
        tds = tr.find_all('span')
        row = [td.text.replace('\n', '').strip() for td in tds]
        rows.append(row)
        rows[-1].append(tr.find('div').text)

    for row in rows:
        if len(row) <= 1:
            rows.remove(row)
    return rows


In [587]:
def cleanRows(type, rows):
    nums = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    if type == 'moneyline':
        for row in rows:
            homeTm = list(row[-1])
            homeTmName = ""
            i = 0
            while i < len(homeTm):
                if homeTm[i] not in nums:
                    homeTmName += homeTm[i]
                    i+=1
                else:
                    break
            row.pop(-1)
            row.insert(0, homeTmName)
            row[1] += row[2]
            row.pop(2)
            i = 0
            while True:
                if i == len(row):
                    break
                if row[i] == 'ML' or row[i] == '':
                    row.pop(i)
                else:
                    i+=1
    elif type == 'spread' or type == 'total':
        for row in rows:
            homeTm = list(row[-1])
            homeTmName = ""
            i = 0
            while i < len(homeTm):
                if homeTm[i] not in nums:
                    homeTmName += homeTm[i]
                    i+=1
                else:
                    break
            row.pop(-1)
            row.insert(0, homeTmName)
            row[1] += row[2]
            row.pop(2)
            i = 0
            while True:
                if i >= len(row):
                    break
                if row[i] == '':
                    row.pop(i)
                elif row[i] == 'N/A':
                    row.insert(i + 1, 'N/A')
                    i+=2
                else:
                    i+=1
    return rows

In [588]:
def toDataFrame(rows, type):
    if type == 'total':
        columns = ['AwayTm', 'HomeTm', 'AwayBestOU', 'AwayBestML', 'HomeBestOU', 'HomeBestML',
                   'AwayPBOU', 'AwayPBML', 'HomePBOU', 'HomePBML',
                   'AwayMGMOU', 'AwayMGMML', 'HomeMGMOU', 'HomeMGMML',
                   'AwayCESOU', 'AwayCESML', 'HomeCESOU', 'HomeBCESML',
                   'AwayFDOU', 'AwayFDML', 'HomeFDOU', 'HomeFDML',
                   'AwayDKOU', 'AwayDKML', 'HomeDKOU', 'HomeDKML',
                   'AwayBROU', 'AwayBRML', 'HomeBROU', 'HomeBRML',
                   'AwaySHOU', 'AwaySHML', 'HomeSHOU', 'HomeSHML',
                   'AwayUBOU', 'AwayUBML', 'HomeUBOU', 'HomeUBML',
                   'Away365OU', 'Away365ML', 'Home365OU', 'Home365ML']
    if type == 'spread':
        columns = ['AwayTm', 'HomeTm', 'AwayBestSpread', 'AwayBestML', 'HomeBestSpread', 'HomeBestML',
                   'AwayPBSpread', 'AwayPBML', 'HomePBSpread', 'HomePBML',
                   'AwayMGMSpread', 'AwayMGMML', 'HomeMGMSpread', 'HomeMGMML',
                   'AwayCESSpread', 'AwayCESML', 'HomeCESSpread', 'HomeBCESML',
                   'AwayFDSpread', 'AwayFDML', 'HomeFDSpread', 'HomeFDML',
                   'AwayDKSpread', 'AwayDKML', 'HomeDKSpread', 'HomeDKML',
                   'AwayBRSpread', 'AwayBRML', 'HomeBRSpread', 'HomeBRML',
                   'AwaySHSpread', 'AwaySHML', 'HomeSHSpread', 'HomeSHML',
                   'AwayUBSpread', 'AwayUBML', 'HomeUBSpread', 'HomeUBML',
                   'Away365Spread', 'Away365ML', 'Home365Spread', 'Home365ML']
    if type == 'moneyline':
        columns = ['AwayTm', 'HomeTm', 'AwayBestML', 'HomeBestML',
                   'AwayPBML', 'HomePBML', 'AwayMGMML', 'HomeMGMML',
                   'AwayCESML', 'HomeBCESML','AwayFDML', 'HomeFDML',
                   'AwayDKML', 'HomeDKML', 'AwayBRML', 'HomeBRML',
                   'AwaySHML', 'HomeSHML', 'AwayUBML', 'HomeUBML',
                   'Away365ML', 'Home365ML']
    df = pd.DataFrame(rows, columns=columns)
    return df

In [589]:
def getSpecificMetric(type):
    data = getData(type)
    soup = convertToSoup(data)
    rows = findData(soup)
    cleanedRows = cleanRows(type, rows)
    df = toDataFrame(cleanedRows, type)
    return df

In [590]:
def getIntervals(type, num, window):
    for i in range(num):
        df = getSpecificMetric(type)
        if i == 0:
            newDf = df
        else:
            newDf = newDf.append(df)
        time.sleep(window)
    return newDf

In [591]:
df = getIntervals('moneyline', 3, 2)

C:\Users\jleus\AppData\Local\Temp\ipykernel_21764\281048714.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jleus\chromedriver.exe")
C:\Users\jleus\AppData\Local\Temp\ipykernel_21764\281048714.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div/div/main/div[2]/div[2]/div[1]/div/div[2]/select/option[3]').click()
C:\Users\jleus\AppData\Local\Temp\ipykernel_21764\281048714.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  rows = driver.find_elements_by_tag_name('tr')
C:\Users\jleus\AppData\Local\Temp\ipykernel_21764\281048714.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jleus\chromedriver.exe")
C:\Users\jleus\AppData\Local\Temp\ipykernel_21764\281048714.py:7: Depre

In [593]:
df

,AwayTm,HomeTm,AwayBestML,HomeBestML,AwayPBML,HomePBML,AwayMGMML,HomeMGMML,AwayCESML,HomeBCESML,...,AwayDKML,HomeDKML,AwayBRML,HomeBRML,AwaySHML,HomeSHML,AwayUBML,HomeUBML,Away365ML,Home365ML
0,NewcastleNEW,West HamWHU,+450,-155,+425,-164,N/A,N/A,N/A,N/A,...,N/A,N/A,+450,-155,+450,-155,+440,-159,+450,-163
1,BrentfordBFC,ArsenalARS,+700,-162,+600,-189,N/A,N/A,N/A,N/A,...,N/A,N/A,+600,-175,+630,-162,+630,-162,+700,-200
2,WatfordWAT,VillaAVFC,+475,-150,+450,-164,N/A,N/A,N/A,N/A,...,N/A,N/A,+475,-150,+475,-150,+460,-157,+475,-163
3,BurnleyBUR,AlbionBRH,+540,-143,+500,-164,N/A,N/A,N/A,N/A,...,N/A,N/A,+525,-150,+540,-143,+525,-157,+500,-154
4,ChelseaCHE,CrystalCRY,-139,+500,-154,+450,N/A,N/A,N/A,N/A,...,N/A,N/A,-148,+475,-139,+490,-139,+490,-167,+500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,BruinsBOS,SenatorsOTT,-175,+176,-185,+155,-175,+145,-180,+155,...,-185,+160,-195,+165,-195,+165,-200,+160,N/A,N/A
41,BluesSTL,Maple LeafsTOR,+180,-200,+175,-220,+165,-200,+180,-220,...,+175,-200,+170,-205,+180,-215,+165,-205,N/A,N/A
42,KingsLA,CoyotesARI,-162,+155,-170,+145,-175,+145,-170,+145,...,-175,+150,-182,+155,-182,+155,-186,+150,N/A,N/A
43,DucksANA,CanucksVAN,+146,-150,+130,-150,+135,-160,+130,-150,...,+135,-155,+135,-157,+135,-157,+132,-162,N/A,N/A
